In [ ]:
!pip install --upgrade -q langchain-openai tiktoken langchain-chroma langchain GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 565.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os


In [ ]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader # 디렉토리에서 파일 로드
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [ ]:
print(os.getcwd())
!mkdir test_repo
os.chdir("test_repo")

/content


In [ ]:
repo_path = os.getcwd()
repo = Repo.clone_from("https://github.com/conceptofmind/toolformer", to_path = repo_path)

In [ ]:
subdirs = [os.path.join(repo_path, d) for d in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, d))]
subdirs

['/content/test_repo/test_repo/test_repo/test_repo/data_handling',
 '/content/test_repo/test_repo/test_repo/test_repo/customToolformer',
 '/content/test_repo/test_repo/test_repo/test_repo/data_generation',
 '/content/test_repo/test_repo/test_repo/test_repo/configs',
 '/content/test_repo/test_repo/test_repo/test_repo/.git',
 '/content/test_repo/test_repo/test_repo/test_repo/flash_attention']

In [ ]:
# Loading Code
loader = GenericLoader.from_filesystem(
    repo_path ,
    glob = "**/*",
    suffixes= [".py"],
    exclude = ["**/non-utf8-encoding.py"],
    parser = LanguageParser(language = Language.PYTHON, parser_threshold = 200),
)
documents = loader.load()

In [ ]:
# document 들을 더 작은 chunk 로 분해
from langchain_text_splitters import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 2000,
    chunk_overlap = 200,
)
texts = python_splitter.split_documents(documents)
len(texts)

121

In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings

db  = Chroma.from_documents(texts, OpenAIEmbeddings(openai_api_key=api_key, disallowed_special = ()))
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 10}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 0.0, openai_api_key=api_key)

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation.",
        ),
    ]
)

retrieval_chain = create_history_aware_retriever(llm, retriever, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)
qa = create_retrieval_chain(retrieval_chain, document_chain)

In [ ]:
questions = ["Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘.",
            "Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어?"]
for question in questions:
    result = qa.invoke({"input": question})
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n\n")


-> **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. 

**Answer**: Toolformer의 Loss는 주어진 문제에 대한 모델의 출력과 실제 정답 간의 차이를 측정하는 방법입니다. 주로 Cross Entropy Loss 또는 Negative Log Likelihood Loss와 같은 손실 함수를 사용하여 계산됩니다.

주어진 코드에서는 Loss를 계산하는 부분이 누락되어 있어 정확한 Loss 함수를 확인할 수 없습니다. 그러나 주로 사용되는 Cross Entropy Loss는 다음과 같이 수식화됩니다:

Loss = -1/N * Σ(y * log(p) + (1-y) * log(1-p))

여기서,
- N은 데이터 포인트의 총 개수
- y는 실제 정답 레이블
- p는 모델의 예측 확률

따라서, 모델이 예측한 확률과 실제 정답 레이블 간의 차이를 최소화하는 방향으로 학습됩니다. Toolformer의 Loss 함수는 모델이 학습하는 목표에 따라 다를 수 있으며, 실제 코드에서 사용된 Loss 함수를 확인하여 더 자세한 정보를 얻을 수 있습니다. 


-> **Question**: Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어? 

**Answer**: Toolformer 클래스에서 사용되는 Tool은 다음과 같습니다:
1. Transformer: Transformer 클래스
2. RotaryEmbedding: RotaryEmbedding 클래스
3. ParallelTransformerBlock: ParallelTransformerBlock 클래스
4. Toolformer: Toolformer 클래스 (자체 정의)
5. RMSNorm: RMSNorm 클래스
6. rotate_half 함수
7. apply_rotary_pos_emb 함수

이러한 Tool들이 Toolformer 클래스 내에서 사용되고 있습니다. 




In [ ]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 문장 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

def answer_quality_score(question: str, answer: str) -> float:
    """질문과 답변 코드의 유사도를 기반으로 품질 점수를 계산하는 함수"""
    # 질문과 답변을 임베딩
    question_embedding = model.encode(question, convert_to_tensor=True)
    answer_embedding = model.encode(answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cos_sim = util.cos_sim(question_embedding, answer_embedding)

    # 유사도 점수를 0~1 사이로 정규화
    normalized_score = float(cos_sim.item()) / 2 + 0.5

    return normalized_score

In [ ]:
# 코드만 가지고 inference 하는 파트
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

subdirs = [os.path.join(repo_path, d) for d in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, d))]

best_answer = ("", 0.0)
best_context = ""
best_subdir = ""
questions = ["Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘.",
            "Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어?"]

for question in questions:
  print(f"\n---------------------- **Question**: {question} ----------------------\n")
  for subdir in tqdm(subdirs):
      loader = GenericLoader.from_filesystem(
          subdir,
          glob="**/*",
          suffixes=[".py"],
          exclude=["**/non-utf8-encoding.py"],
          parser=LanguageParser(language=Language.PYTHON, parser_threshold=200),
      )
      documents = loader.load()

      python_splitter = RecursiveCharacterTextSplitter.from_language(
          language=Language.PYTHON,
          chunk_size=1000,
          chunk_overlap=100,
      )
      texts = python_splitter.split_documents(documents)
      if not texts:
        pass
      else:
        db = Chroma.from_documents(texts,
                                  OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                  collection_metadata = {'hnsw:space': 'cosine'})


      retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

      prompt = ChatPromptTemplate.from_messages([
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
          ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
      ])

      retrieval_chain = create_history_aware_retriever(llm, retriever, prompt)

      prompt = ChatPromptTemplate.from_messages([
          ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
      ])


      document_chain = create_stuff_documents_chain(llm, prompt)
      qa = create_retrieval_chain(retrieval_chain, document_chain)

      # print results
      result = qa.invoke({"input": question})
      answer = result['answer']
      context = result['context']
      score = answer_quality_score(question, answer)

      # update score, best answer, best subdir
      if score > best_answer[1]:
          best_answer = (answer, score)
          best_subdir = subdir
          best_context = context
      print(f"\n CURRENT SUBDIRECTORY : {subdir}")
      print(f"**Answer**: {answer} \n\n**Score**: {score}\n\n")

  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")
  print(f"\n -> **Question**: {question} \n")
  print(f"**Best Answer**: {best_answer[0]} \n\n**Score**: {best_answer[1]}\n\n")
  print(f"**Best Subdirectory**: {best_subdir}")
  print(f"**Best Context**:\n")
  print(best_context)
  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")



---------------------- **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. ----------------------



 17%|█▋        | 1/6 [00:06<00:33,  6.65s/it]


 CURRENT SUBDIRECTORY : /content/test_repo/configs
**Answer**: Toolformer의 Loss는 주어진 데이터셋에 대한 모델의 예측과 실제 정답 간의 차이를 측정하는 지표입니다. Loss는 모델이 얼마나 잘 작동하는지를 평가하는 중요한 지표 중 하나이며, 최소화되어야 하는 목표입니다.

Loss는 주로 Cross-Entropy Loss 함수를 사용하여 계산됩니다. Cross-Entropy Loss 함수는 다음과 같이 정의됩니다:

\[ \text{Cross-Entropy Loss} = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(p_{i,c}) \]

여기서,
- \( N \)은 데이터 포인트의 총 수,
- \( C \)는 클래스의 수,
- \( y_{i,c} \)는 실제 정답 레이블 (0 또는 1)로, \( i \)번째 데이터 포인트가 \( c \)번째 클래스에 속하는지 여부를 나타냅니다,
- \( p_{i,c} \)는 모델의 예측 확률로, \( i \)번째 데이터 포인트가 \( c \)번째 클래스에 속할 확률을 나타냅니다.

이 Loss 함수는 모델의 예측이 실제 정답과 얼마나 일치하는지를 측정하며, 모델이 더 나은 예측을 하도록 학습하도록 도와줍니다.

이 답변은 주어진 맥락을 기반으로 추론하여 제공되었습니다. 

**Score**: 0.692680835723877




 33%|███▎      | 2/6 [00:10<00:19,  4.91s/it]


 CURRENT SUBDIRECTORY : /content/test_repo/.git
**Answer**: 주어진 문맥에서는 Toolformer의 Loss에 대한 구체적인 정보나 수식이 제공되지 않습니다. 따라서 Loss 함수에 대한 자세한 내용은 이 문서에서는 확인할 수 없습니다. Loss 함수는 모델이 학습 중에 예측과 실제 값 사이의 차이를 측정하는 함수이며, 일반적으로 평균 제곱 오차(Mean Squared Error)나 교차 엔트로피(Cross Entropy)와 같은 함수가 사용됩니다. 이러한 Loss 함수는 모델이 얼마나 잘 학습되고 있는지를 판단하는 중요한 지표입니다. 

**Score**: 0.7822863459587097




...


 CURRENT SUBDIRECTORY : /content/test_repo/flash_attention
**Answer**: Toolformer가 사용하는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공되었습니다. 

**Score**: 0.8577778041362762




 83%|████████▎ | 5/6 [00:11<00:02,  2.39s/it]


 CURRENT SUBDIRECTORY : /content/test_repo/data_handling
**Answer**: Toolformer에서 사용되는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공됩니다. 

**Score**: 0.8643501400947571




100%|██████████| 6/6 [00:14<00:00,  2.42s/it]


 CURRENT SUBDIRECTORY : /content/test_repo/data_generation
**Answer**: Toolformer 모델이 사용하는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. (출처: Toolformer: Language Models Can Teach Themselves to Use Tools) 

**Score**: 0.8861420154571533


--------------------------------------------------------------------------
--------------------------------------------------------------------------

 -> **Question**: Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어? 

**Best Answer**: Toolformer 모델이 사용하는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. (출처: Toolformer: Language Models Can Teach Themselves to Use Tools) 

**Score**: 0.8861420154571533


**Best Subdirectory**: /content/test_repo/data_generation
**Best Context**:

[Document(page_content="Toolformer: Language Models Can Teach Themselves to Use Tools\nTimo Schick Jane Dwivedi-Yu Roberto Dessì†Roberta Raileanu\nMaria Lomeli Luke Zettlemoyer Nicola Cancedda Thomas Scialom\nMeta AI Research†Universitat Pompeu Fabra\nAbstract\nLanguage models

In [ ]:
best_context #어디를 참고했는지

[Document(page_content="Toolformer: Language Models Can Teach Themselves to Use Tools\nTimo Schick Jane Dwivedi-Yu Roberto Dessì†Roberta Raileanu\nMaria Lomeli Luke Zettlemoyer Nicola Cancedda Thomas Scialom\nMeta AI Research†Universitat Pompeu Fabra\nAbstract\nLanguage models (LMs) exhibit remarkable\nabilities to solve new tasks from just a few\nexamples or textual instructions, especially at\nscale. They also, paradoxically, struggle with\nbasic functionality, such as arithmetic or fac-\ntual lookup, where much simpler and smaller\nmodels excel. In this paper, we show that\nLMs can teach themselves to use external tools\nvia simple APIs and achieve the best of both\nworlds. We introduce Toolformer , a model\ntrained to decide which APIs to call, when to\ncall them, what arguments to pass, and how to\nbest incorporate the results into future token\nprediction. This is done in a self-supervised\nway, requiring nothing more than a handful of\ndemonstrations for each API. We incorporate

## answer from pdf

In [ ]:
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
pdf = PyPDFLoader("/content/2302.04761v1 (4).pdf")
pages = pdf.load_and_split()

In [ ]:
import numpy as np
import random

def answer_quality_score_multiple(question: str, context_code, context_pdf, weight_qc = 0.2, weight_qp = 0.35, weight_cp = 0.45):
    """질문과 답변 코드의 유사도를 기반으로 품질 점수를 계산하는 함수"""
    # embedding
    question_embedding = model.encode(question, convert_to_tensor=True)
    context_code_embedding = model.encode(context_code, convert_to_tensor=True)
    context_pdf_embedding = model.encode(context_pdf, convert_to_tensor=True)

    cos_sim_qc = util.cos_sim(question_embedding, context_code_embedding)
    cos_sim_qp = util.cos_sim(question_embedding, context_pdf_embedding)
    cos_sim_cp = util.cos_sim(context_code_embedding, context_pdf_embedding)

    normalized_score = float(cos_sim_qc.item()) * weight_qc + float(cos_sim_qp.item()) * weight_qp + float(cos_sim_cp.item()) * weight_cp

    return normalized_score

# def random_search(cos_sim_qc, cos_sim_qp, cos_sim_cp, num_iterations=100):
#   best_weights = None
#   best_score = -float('inf')

#   for _ in range(num_iterations):
#     weight_qc = random.random()
#     weight_qp = random.random()
#     weight_cp = 1 - weight_qc - weight_qp

#     normalized_score = float(cos_sim_qc.item()) * weight_qc + float(cos_sim_qp.item()) * weight_qp + float(cos_sim_cp.item()) * weight_cp

#     if normalized_score > best_score:
#       best_weights = (weight_qc, weight_qp, weight_cp)
#       best_score = normalized_score

#   return best_weights, best_score

In [ ]:
!pip install -q transformers

In [ ]:
import numpy as np

def generate_weight_combinations(iter):
    combinations = []
    for _ in range(iter):
        w1 = np.random.rand()
        w2 = np.random.rand() * (1 - w1)
        w3 = 1 - w1 - w2
        combinations.append((w1, w2, w3))
    return combinations

iter = 10
combinations = generate_weight_combinations(iter)
for i, combo in enumerate(combinations):
    print(f"Combination {i+1}: {combo}")

Combination 1: (0.6458414485054641, 0.3031001124404972, 0.05105843905403867)
Combination 2: (0.5809631519598101, 0.21589852467844373, 0.2031383233617462)
Combination 3: (0.377493951373875, 0.006955407191395286, 0.6155506414347297)
Combination 4: (0.7344643333908267, 0.1286782882793024, 0.13685737832987088)
Combination 5: (0.9601382046237106, 0.01732343690685467, 0.022538358469434753)
Combination 6: (0.07239842745791991, 0.32680104617428773, 0.6008005263677924)
Combination 7: (0.5990802656544729, 0.1142454171831855, 0.2866743171623416)
Combination 8: (0.0400550103501256, 0.225231345951922, 0.7347136436979524)
Combination 9: (0.711373743808123, 0.24429538106594947, 0.04433087512592751)
Combination 10: (0.8319061800321214, 0.006649174977228679, 0.16144464499064998)


In [ ]:
# TRIAL
subdirs = [os.path.join(repo_path, d) for d in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, d))]

questions = ["Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘.",
            "Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어?"]

for question in questions:
  best_answer = ("", 0.0)
  best_subdir = ""
  best_context_code = ""
  best_context_pdf = ""

  qa_code = []
  result_code =[]
  answer_code =[]
  context_code = []
  print(f"\n---------------------- **Question**: {question} ----------------------\n")
  for i, subdir in tqdm(enumerate(subdirs)):
    # code load setup
    loader = GenericLoader.from_filesystem(
        repo_path + "/data_generation" ,
        glob = "**/*",
        suffixes= [".py"],
        exclude = ["**/non-utf8-encoding.py"],
        parser = LanguageParser(language = Language.PYTHON, parser_threshold = 200),
    )
    python_splitter = RecursiveCharacterTextSplitter.from_language(
        language = Language.PYTHON,
        chunk_size = 1000,
        chunk_overlap = 100,
    )

    # code load
    documents_code = loader.load()
    texts_code = python_splitter.split_documents(documents_code)

    db_code = Chroma.from_documents(texts_code,
                                OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                collection_metadata = {'hnsw:space': 'cosine'})

    retriever_code = db_code.as_retriever(search_type="mmr", search_kwargs={"k": 5})

    # prompt
    prompt = ChatPromptTemplate.from_messages([
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
    ])
    retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
    # retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ])
    document_chain = create_stuff_documents_chain(llm, prompt)

    # result code
    qa_code = create_retrieval_chain(retrieval_chain_code, document_chain)
    result_code[i] = qa_code.invoke({"input": question})
    answer_code[i] = result_code[i]['answer']
    context_code[i] = result_code[i]['context']


  # pdf load
  pdf = PyPDFLoader("/content/2302.04761v1 (4).pdf")
  pages = pdf.load_and_split()

  db_pdf = Chroma.from_documents(pages,
                              OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                              collection_metadata = {'hnsw:space': 'cosine'})

  retriever_pdf = db_pdf.as_retriever(search_type="mmr", search_kwargs={"k": 5})

  # prompt
  prompt = ChatPromptTemplate.from_messages([
      ("placeholder", "{chat_history}"),
      ("user", "{input}"),
      ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
  ])
  # retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
  retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

  prompt = ChatPromptTemplate.from_messages([
      ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
      ("placeholder", "{chat_history}"),
      ("user", "{input}"),
  ])
  document_chain = create_stuff_documents_chain(llm, prompt)

  # result pdf
  qa_pdf = create_retrieval_chain(retrieval_chain_pdf, document_chain)
  result_pdf = qa_pdf.invoke({"input": question})
  answer_pdf = result_pdf['answer']
  context_pdf = result_pdf["context"]

  for i in range(len(subdirs)):
    final_score = answer_quality_score_multiple(question, str(context_code[i]), str(context_pdf[i]), 0.2, 0.3, 0.5)
    if final_score > best_answer[1]:
      best_answer_code = (answer_code[i], final_score)
      best_answer_pdf = (answer_pdf, final_score)
      best_subdir = subdir
      best_context_pdf = context_pdf
      best_context_code = context_code[i]

    # print(f"-> **Question**: {question} \n")
    # print(f"**Code 에 대한 대답**: {answer_code} \n\n")
    # print(f"**PDF 에 대한 대답**: {answer_pdf} \n\n")
    # print(f"**Score**: {final_score}\n\n")


  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")
  print(f"\n -> **Question**: {question} \n")
  print(f"**Best Score** : {best_answer_code[1]}")
  print(f"**Best Answer CODE**: {best_answer_code[0]} \n\n")
  print(f"**Best Answer PDF**: {best_answer_pdf[0]} \n\n")
  print(f"**Best Subdirectory**: {best_subdir}")
  print("**Best Contex CODE**:\n")
  print(f"```python\n{best_context_code}\n```")  # Print context as a formatted code block
  print("**Best Contex PDF**:\n")
  print(f"\n{best_context_pdf}\n")  # Print context as a formatted code block
  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")



---------------------- **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. ----------------------



100%|██████████| 6/6 [01:24<00:00, 14.09s/it]


--------------------------------------------------------------------------
--------------------------------------------------------------------------

 -> **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. 

**Best Score** : 0.6199304983019829
**Best Answer CODE**: Toolformer의 Loss는 주어진 데이터셋에 대한 모델의 예측과 실제 정답 간의 차이를 측정하는 지표입니다. Loss는 모델이 얼마나 잘 작동하는지를 평가하는 중요한 지표 중 하나이며, 최소화되어야 하는 목표입니다.

Loss는 주로 모델의 출력과 실제 정답 간의 차이를 계산하는 손실 함수를 통해 계산됩니다. 손실 함수는 모델이 예측한 값과 실제 값 사이의 거리를 측정하고, 이 거리를 최소화하는 방향으로 모델을 학습시키는 역할을 합니다.

Toolformer의 Loss는 다양한 요소에 따라 상이할 수 있지만, 일반적으로는 모델이 예측한 값과 실제 값 사이의 차이를 계산하는 함수를 통해 계산됩니다. 이러한 Loss 함수는 주어진 데이터셋과 모델의 목적에 따라 다를 수 있습니다.

위의 문맥에서는 Loss에 대한 구체적인 수식이 제공되지 않았기 때문에, 해당 모델의 Loss 함수에 대한 자세한 내용은 문맥에서 확인할 수 없습니다. 


**Best Answer PDF**: Toolformer의 Loss는 주어진 데이터셋에 대한 모델의 예측과 실제 정답 간의 차이를 측정하는 지표입니다. Loss를 계산하는 방법은 일반적으로 모델의 출력과 실제 정답 간의 거리를 나타내는 손실 함수를 사용하여 수행됩니다. 

위의 텍스트에서는 Loss에 대한 구체적인 수식은 제시되지 않았습니다. 하지만 Loss를 계산하는 일반적인 방법은 모델의 출력과 실제 정답 간의 차이를 측정하는 손실

100%|██████████| 6/6 [00:50<00:00,  8.35s/it]

--------------------------------------------------------------------------
--------------------------------------------------------------------------

 -> **Question**: Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어? 

**Best Score** : 0.678546816110611
**Best Answer CODE**: Toolformer에서 사용되는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공되었습니다. 


**Best Answer PDF**: Toolformer에서 사용되는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공됩니다. 


**Best Subdirectory**: /content/test_repo/data_generation
**Best Contex CODE**:

```python
[Document(page_content="Toolformer: Language Models Can Teach Themselves to Use Tools\nTimo Schick Jane Dwivedi-Yu Roberto Dessì†Roberta Raileanu\nMaria Lomeli Luke Zettlemoyer Nicola Cancedda Thomas Scialom\nMeta AI Research†Universitat Pompeu Fabra\nAbstract\nLanguage models (LMs) exhibit remarkable\nabilit

In [ ]:
str(result_code["context"][4])

"page_content='Model T EMPLAMA D ATESET\\nGPT-J 13.7 3.9\\nGPT-J + CC 12.9 2.9\\nToolformer (disabled) 12.7 5.9\\nToolformer 16.3 27.3\\nOPT (66B) 14.5 1.3\\nGPT-3 (175B) 15.5 0.8\\nTable 7: Results for the temporal datasets. Toolformer\\noutperforms all baselines, but does not make use of the\\ncalendar tool for T EMPLAMA.\\nFor both tasks, we use the same evaluation as for\\nthe original LAMA dataset.\\nResults shown in Table 7 illustrate that Tool-\\nformer outperforms all baselines for both TEM-\\nPLAMA andDATESET . However, closer inspec-\\ntion shows that improvements on TEMPLAMA\\ncan not be attributed to the calendar tool, which is\\nonly used for 0.2% of all examples, but mostly to\\nthe Wikipedia search and question answering tools,\\nwhich Toolformer calls the most. This makes sense\\ngiven that named entities in TEMPLAMA are often\\nso speciﬁc and rare that even knowing the exact\\ndate alone would be of little help. The best course\\nof action for this dataset – ﬁrst query

In [ ]:
result_code["answer"]

'Toolformer에서 사용되는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공되었습니다.'

In [ ]:
# 최종 결과
subdirs = [os.path.join(repo_path, d) for d in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, d))]
subdirs.append("/content/test_repo") # 자기 자신 폴더도 확인되게끔 수정

questions = ["Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘.",
            "Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어?"]

for question in questions:
  best_score = 0.0
  best_answer = ("")
  best_subdir = ""
  best_context_code = ""
  best_context_pdf = ""

  weight_combinations = generate_weight_combinations(200)
  print(f"\n---------------------- **Question**: {question} ----------------------\n")
  for subdir in tqdm(subdirs):
    # code load setup
    loader = GenericLoader.from_filesystem(
        subdir,
        glob = "**/*",
        suffixes= [".py"],
        exclude = ["**/non-utf8-encoding.py"],
        parser = LanguageParser(language = Language.PYTHON, parser_threshold = 200),
    )
    python_splitter = RecursiveCharacterTextSplitter.from_language(
        language = Language.PYTHON,
        chunk_size = 4000,
        chunk_overlap = 100,
    )

    # code load
    documents_code = loader.load()
    texts_code = python_splitter.split_documents(documents_code)

    # 만약 .py 가 없는 빈 폴더면 error 가 나기 때문에 error handling
    try:
      db_code = Chroma.from_documents(texts_code,
                                  OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                  collection_metadata = {'hnsw:space': 'cosine'},
                                      collection_name = "code")
      retriever_code = db_code.as_retriever(search_type="mmr", search_kwargs={"k": 2})

      # prompt
      prompt = ChatPromptTemplate.from_messages([
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
          ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
      ])
      retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
      # retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

      prompt = ChatPromptTemplate.from_messages([
          ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
      ])
      document_chain = create_stuff_documents_chain(llm, prompt)

      # result code
      qa_code = create_retrieval_chain(retrieval_chain_code, document_chain)
      result_code = qa_code.invoke({"input": question})
      answer_code = result_code['answer']
      context_code = result_code['context']

    except ValueError as e:
      print(f"Skipping subdir {subdir} due to error: {e}")
      continue


    # pdf load
    pdf = PyPDFLoader("/content/2302.04761v1 (4).pdf")
    pages = pdf.load_and_split()

    db_pdf = Chroma.from_documents(pages,
                                OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                collection_metadata = {'hnsw:space': 'cosine'},
                                   collection_name = "pdf") # collection_name 을 설정해주지 않으면

    retriever_pdf = db_pdf.as_retriever(search_type="mmr", search_kwargs={"k": 3})

    # prompt
    prompt = ChatPromptTemplate.from_messages([
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
    ])
    # retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
    retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ])
    document_chain = create_stuff_documents_chain(llm, prompt)

    # result pdf
    qa_pdf = create_retrieval_chain(retrieval_chain_pdf, document_chain)
    result_pdf = qa_pdf.invoke({"input": question})
    answer_pdf = result_pdf['answer']
    context_pdf = result_pdf["context"]

    context_code_str = str(context_code)
    context_pdf_str = str(context_pdf)
    for i, weights in enumerate(weight_combinations):
        final_score = answer_quality_score_multiple(question, context_code_str, context_pdf_str, *weights)
        if final_score > best_score:
            best_score = final_score # final score update to better score
            best_answer_code = (answer_code, final_score)
            best_answer_pdf = (answer_pdf, final_score)
            best_subdir = subdir
            best_context_pdf = context_pdf
            best_context_code = context_code
            best_weights = weights
        if i % 50 == 0:
                        print(f"Iteration {i}: Current best score: {best_score} &&& Best Weights {i} : {best_weights}")


    # print(f"-> **Question**: {question} \n")
    # print(f"**Code 에 대한 대답**: {answer_code} \n\n")
    # print(f"**PDF 에 대한 대답**: {answer_pdf} \n\n")
    # print(f"**Score**: {final_score}\n\n")


  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")
  print(f"\n -> **Question**: {question} \n")
  print(f"**Best Score** : {best_answer_code[1]}")
  print(f"**Best Answer CODE**: {best_answer_code[0]} \n\n")
  print(f"**Best Answer PDF**: {best_answer_pdf[0]} \n\n")
  print(f"**Best Subdirectory**: {best_subdir}")
  print("**Best Contex CODE**:\n")
  print(f"```python\n{best_context_code}\n```")  # Print context as a formatted code block
  print("**Best Contex PDF**:\n")
  print(f"\n{best_context_pdf}\n")  # Print context as a formatted code block
  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")



---------------------- **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. ----------------------



 14%|█▍        | 1/7 [00:00<00:00,  7.16it/s]

Skipping subdir /content/test_repo/configs due to error: Expected IDs to be a non-empty list, got 0 IDs


 29%|██▊       | 2/7 [00:00<00:00,  6.70it/s]

Skipping subdir /content/test_repo/.git due to error: Expected IDs to be a non-empty list, got 0 IDs


...

Iteration 0: Current best score: 0.4573482313929848 &&& Best Weights 0 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 50: Current best score: 0.4573482313929848 &&& Best Weights 50 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 100: Current best score: 0.4573482313929848 &&& Best Weights 100 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 150: Current best score: 0.4573482313929848 &&& Best Weights 150 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)


 86%|████████▌ | 6/7 [03:46<00:56, 56.83s/it]

Iteration 0: Current best score: 0.4573482313929848 &&& Best Weights 0 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 50: Current best score: 0.4573482313929848 &&& Best Weights 50 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 100: Current best score: 0.4573482313929848 &&& Best Weights 100 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)
Iteration 150: Current best score: 0.4573482313929848 &&& Best Weights 150 : (0.008551616023304232, 0.1232970923419235, 0.8681512916347722)


100%|██████████| 7/7 [05:01<00:00, 43.07s/it]

--------------------------------------------------------------------------
--------------------------------------------------------------------------

 -> **Question**: Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어? 

**Best Score** : 0.4573482313929848
**Best Answer CODE**: Toolformer에서 사용되는 도구들은 다음과 같습니다:
- torch
- transformers
- langchain_llmchain
- llmchain_prompt
- typing
- base_api

이 정보는 주어진 코드에서 가져온 것입니다. 


**Best Answer PDF**: Toolformer가 사용하는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 언급되었습니다. 


**Best Subdirectory**: /content/test_repo/flash_attention
**Best Contex CODE**:

```python
[Document(page_content='import torch\nfrom transformers import (\n    PreTrainedTokenizerBase,\n    PreTrainedModel,\n)\nfrom tools import langchain_llmchain\nfrom prompts import llmchain_prompt\nfrom typing import List\nfrom data_generation.base_api import APICallPostprocessing\n\n\n\n# TODO: Per API?\nMAX_BATCH_SIZE =

# 위에가 잘 안되는 이유

Weighted Sum 을 하나의 component 가 dominate 함. score 를 내기 더 쉬운 방향으로만 학습하려고 하지, 우리가 의도한 올바른 학습방향으로 학습하려고 하지 않음.

- 해결방안 1 : 클러스터 자체를 바꿔야 함. context_pdf 와 context_code term 이 final_score 를 dominate 하므로, 3가지를 균형있게 밸런싱 할 수 있는 새로운 클러스터(객체) 가 필요.

  - 혹은 클러스터를 더 잘게 쪼개보는 방법. (당연한것. 현재 모든 객체를 고려하지 못함. - `answer_pdf`, `answer_code` 객체가 없음.)

- 해결방안 2 : 아니면 metric 을 바꿔야 함.

In [ ]:
len(weight_combinations)

1000

In [ ]:
context_code

[Document(page_content='class Toolformer(nn.Module):\n    def __init__(\n        self,\n        dim,\n        num_tokens,\n        depth,\n        dim_head=64,\n        heads=8,\n        ff_mult=4,\n    ):\n        super().__init__()\n\n        self.emb = nn.Embedding(num_tokens, dim)\n        self.transformer = Transformer(dim, depth, heads, dim_head, ff_mult)\n        self.to_logits = nn.Linear(dim, num_tokens)\n\n    def forward(self, x):\n        x = self.emb(x)\n        x = self.transformer(x)\n        x = self.to_logits(x)\n        return x', metadata={'content_type': 'functions_classes', 'language': 'python', 'source': '/content/test_repo/gptj_pytorch.py'}),
 Document(page_content='class Toolformer(nn.Module):\n    def __init__(\n        self,\n        dim,\n        num_tokens,\n        depth,\n        dim_head=64,\n        heads=8,\n        ff_mult=4,\n    ):\n        super().__init__()\n\n        self.emb = nn.Embedding(num_tokens, dim)\n        self.transformer = Transformer(

In [ ]:
answer_pdf

'Toolformer가 사용하는 도구에는 계산기, 질의응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. 이 정보는 논문 "Toolformer: Language Models Can Teach Themselves to Use Tools"에서 제공되었습니다.'

In [ ]:
weight_combinations = generate_weight_combinations(1000)
weight_combinations[1]

(0.15870159635633874, 0.26677729606427003, 0.5745211075793912)

# Version 3


In [ ]:
def answer_quality_score_quintuple(question: str, context_code, context_pdf, answer_code, answer_pdf,
                                   weight_qc=0.1, weight_qp=0.1, weight_qac=0.1, weight_qap=0.1, weight_cac=0.1,
                                   weight_cap=0.1, weight_cpdf=0.1, weight_pap=0.1, weight_capdf=0.1, weight_cp=0.1):
    # Embedding
    question_embedding = model.encode(question, convert_to_tensor=True)
    context_code_embedding = model.encode(context_code, convert_to_tensor=True)
    context_pdf_embedding = model.encode(context_pdf, convert_to_tensor=True)
    answer_code_embedding = model.encode(answer_code, convert_to_tensor=True)
    answer_pdf_embedding = model.encode(answer_pdf, convert_to_tensor=True)

    # cosine simliarity
    cos_sim_qc = util.cos_sim(question_embedding, context_code_embedding)
    cos_sim_qp = util.cos_sim(question_embedding, context_pdf_embedding)
    cos_sim_qac = util.cos_sim(question_embedding, answer_code_embedding)
    cos_sim_qap = util.cos_sim(question_embedding, answer_pdf_embedding)
    cos_sim_cac = util.cos_sim(context_code_embedding, answer_code_embedding)
    cos_sim_cap = util.cos_sim(context_code_embedding, answer_pdf_embedding)
    cos_sim_cpdf = util.cos_sim(context_code_embedding, context_pdf_embedding)
    cos_sim_pap = util.cos_sim(context_pdf_embedding, answer_pdf_embedding)
    cos_sim_capdf = util.cos_sim(context_code_embedding, context_pdf_embedding)
    cos_sim_cp = util.cos_sim(context_code_embedding, context_pdf_embedding)

    # weighted sum of score
    normalized_score = (float(cos_sim_qc.item()) * weight_qc + float(cos_sim_qp.item()) * weight_qp +
                        float(cos_sim_qac.item()) * weight_qac + float(cos_sim_qap.item()) * weight_qap +
                        float(cos_sim_cac.item()) * weight_cac + float(cos_sim_cap.item()) * weight_cap +
                        float(cos_sim_cpdf.item()) * weight_cpdf + float(cos_sim_pap.item()) * weight_pap +
                        float(cos_sim_capdf.item()) * weight_capdf + float(cos_sim_cp.item()) * weight_cp)

    return normalized_score

In [ ]:
def generate_weight_combinations_10(iter):
    combinations = []
    for _ in range(iter):
        weights = np.random.rand(10)
        weights /= weights.sum()  # Normalize the weights so that their sum is 1
        combinations.append(tuple(weights))
    return combinations

weight_combinations = generate_weight_combinations_10(3)
for weights in weight_combinations:
    print(weights)

(0.1492176761662531, 0.12558587951920738, 0.12138937726289858, 0.15165358259387174, 0.02300107715495243, 0.02215443035693413, 0.027779184781749264, 0.10544386298898391, 0.10132429970211483, 0.17245062947303463)
(0.07067470904833198, 0.11781920969103751, 0.13208277390755996, 0.005360342802557969, 0.0656387865377058, 0.07502715008402899, 0.17304823682644624, 0.15256441797620823, 0.15572730158092848, 0.05205707154519494)
(0.11820045043024718, 0.03125748285552181, 0.17833501118426026, 0.16037076044423199, 0.08653647978247064, 0.07310523938181737, 0.0958217004051037, 0.02141444296057042, 0.07074221781624905, 0.16421621473952758)


In [ ]:
subdirs.append("/content/test_repo")

In [ ]:
repo_path

'/content/test_repo'

In [ ]:
subdirs = [os.path.join(repo_path, d) for d in os.listdir(repo_path) if os.path.isdir(os.path.join(repo_path, d))]
subdirs.append("/content/test_repo") # 자기 자신 폴더도 확인되게끔 수정

questions = ["Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘.",
            "Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어?"]

for question in questions:
  best_score = 0.0
  best_answer = ("")
  best_subdir = ""
  best_context_code = ""
  best_context_pdf = ""

  weight_combinations = generate_weight_combinations_10(200)
  print(f"\n---------------------- **Question**: {question} ----------------------\n")

  for i, weights in enumerate(tqdm(weight_combinations, miniters=int(round(len(weight_combinations)/100)))):
    for subdir in subdirs:
      # code load setup
      loader = GenericLoader.from_filesystem(
          subdir,
          glob = "**/*",
          suffixes= [".py"],
          exclude = ["**/non-utf8-encoding.py"],
          parser = LanguageParser(language = Language.PYTHON, parser_threshold = 200),
      )
      python_splitter = RecursiveCharacterTextSplitter.from_language(
          language = Language.PYTHON,
          chunk_size = 4000,
          chunk_overlap = 100,
      )

      # code load
      documents_code = loader.load()
      texts_code = python_splitter.split_documents(documents_code)

      # 만약 .py 가 없는 빈 폴더면 error 가 나기 때문에 error handling
      try:
        db_code = Chroma.from_documents(texts_code,
                                    OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                    collection_metadata = {'hnsw:space': 'cosine'},
                                        collection_name = "code")
        retriever_code = db_code.as_retriever(search_type="mmr", search_kwargs={"k": 2})

        # prompt
        prompt = ChatPromptTemplate.from_messages([
            ("placeholder", "{chat_history}"),
            ("user", "{input}"),
            ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
        ])
        retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
        # retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

        prompt = ChatPromptTemplate.from_messages([
            ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
            ("placeholder", "{chat_history}"),
            ("user", "{input}"),
        ])
        document_chain = create_stuff_documents_chain(llm, prompt)

        # result code
        qa_code = create_retrieval_chain(retrieval_chain_code, document_chain)
        result_code = qa_code.invoke({"input": question})
        answer_code = result_code['answer']
        context_code = result_code['context']

      except ValueError as e:
        pass
        # print(f"Skipping subdir {subdir} due to error: {e}")
        continue


      # pdf load
      pdf = PyPDFLoader("/content/2302.04761v1 (4).pdf")
      pages = pdf.load_and_split()

      db_pdf = Chroma.from_documents(pages,
                                  OpenAIEmbeddings(openai_api_key=api_key, disallowed_special=()),
                                  collection_metadata = {'hnsw:space': 'cosine'},
                                    collection_name = "pdf") # collection_name 을 설정해주지 않으면

      retriever_pdf = db_pdf.as_retriever(search_type="mmr", search_kwargs={"k": 2})

      # prompt
      prompt = ChatPromptTemplate.from_messages([
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
          ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation."),
      ])
      # retrieval_chain_code = create_history_aware_retriever(llm, retriever_code, prompt)
      retrieval_chain_pdf = create_history_aware_retriever(llm, retriever_pdf, prompt)

      prompt = ChatPromptTemplate.from_messages([
          ("system", "Answer the user's question based on the below context : \n\n{context}. By each answer, should always give where the answer's source came from. If you can't find the answer, you should do the inference and give me the answer."),
          ("placeholder", "{chat_history}"),
          ("user", "{input}"),
      ])
      document_chain = create_stuff_documents_chain(llm, prompt)

      # result pdf
      qa_pdf = create_retrieval_chain(retrieval_chain_pdf, document_chain)
      result_pdf = qa_pdf.invoke({"input": question})
      answer_pdf = result_pdf['answer']
      context_pdf = result_pdf["context"]

      context_code_str = str(context_code)
      context_pdf_str = str(context_pdf)

      # CALCULATE the final score
      final_score = answer_quality_score_quintuple(question, context_code_str, context_pdf_str, answer_code, answer_pdf, *weights)
      if final_score > best_score:
          best_score = final_score # final score update to better score
          best_answer_code = (answer_code, final_score)
          best_answer_pdf = (answer_pdf, final_score)
          best_subdir = subdir
          best_context_pdf = context_pdf
          best_context_code = context_code
          best_weights = weights

    if i % 10 == 0:
      print(f"Iteration {i}: Current best score: {best_score} &&& Best Weights {i} : {best_weights}")


  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")
  print(f"\n -> **Question**: {question} \n")
  print(f"**Best Score** : {best_answer_code[1]}")
  print(f"**Best Answer CODE**: {best_answer_code[0]} \n\n")
  print(f"**Best Answer PDF**: {best_answer_pdf[0]} \n\n")
  print(f"**Best Subdirectory**: {best_subdir}")
  print("**Best Contex CODE**:\n")
  print(f"```python\n{best_context_code}\n```")  # Print context as a formatted code block
  print("**Best Contex PDF**:\n")
  print(f"\n{best_context_pdf}\n")  # Print context as a formatted code block
  print("--------------------------------------------------------------------------")
  print("--------------------------------------------------------------------------")



---------------------- **Question**: Toolformer 의 Loss 는 어떻게 상정되는지 Loss 의 수식을 중심으로 알려줘. ----------------------



  0%|          | 1/200 [00:52<2:55:11, 52.82s/it]

Iteration 0: Current best score: 0.3628209783691579 &&& Best Weights 0 : (0.0852772530223539, 0.15608551493568135, 0.0022446145603125765, 0.13691004128762826, 0.0877927546339296, 0.05277234222440326, 0.08112802382357454, 0.1500090008945868, 0.14881835744910274, 0.09896209716842692)


  6%|▌         | 11/200 [09:48<2:49:54, 53.94s/it]

Iteration 10: Current best score: 0.43022003265777364 &&& Best Weights 10 : (0.11952944023749872, 0.06646392642573881, 0.1798993747710065, 0.10143256134366915, 0.0742174261464129, 0.0030760379730034246, 0.0661481786552112, 0.07338172903273636, 0.2086029765719843, 0.10724834884273865)


...

Iteration 180: Current best score: 0.536230932574046 &&& Best Weights 180 : (0.012183247552104256, 0.10026529640555645, 0.18151324758119516, 0.17073438496545748, 0.10502537024077112, 0.01704885446973744, 0.06331769582366116, 0.15275527427484964, 0.07617036169436044, 0.1209862669923068)


 96%|█████████▌| 191/200 [1:27:11<03:57, 26.40s/it]

Iteration 190: Current best score: 0.536230932574046 &&& Best Weights 190 : (0.012183247552104256, 0.10026529640555645, 0.18151324758119516, 0.17073438496545748, 0.10502537024077112, 0.01704885446973744, 0.06331769582366116, 0.15275527427484964, 0.07617036169436044, 0.1209862669923068)


100%|██████████| 200/200 [1:31:04<00:00, 27.32s/it]

--------------------------------------------------------------------------
--------------------------------------------------------------------------

 -> **Question**: Toolformer 에서 사용하고 있는 Tool 에는 어떤게 있어? 

**Best Score** : 0.536230932574046
**Best Answer CODE**: Toolformer 클래스에서 사용되는 Tool은 nn.Embedding, Transformer, nn.Linear 입니다. (위 코드 참조) 


**Best Answer PDF**: Toolformer 모델이 사용하는 도구에는 계산기, 질문 응답 시스템, 검색 엔진, 번역 시스템 및 캘린더가 포함되어 있습니다. (Source: Toolformer: Language Models Can Teach Themselves to Use Tools) 


**Best Subdirectory**: /content/test_repo/data_generation
**Best Contex CODE**:

```python
[Document(page_content='class Toolformer(nn.Module):\n    def __init__(\n        self,\n        dim,\n        num_tokens,\n        depth,\n        dim_head=64,\n        heads=8,\n        ff_mult=4,\n    ):\n        super().__init__()\n\n        self.emb = nn.Embedding(num_tokens, dim)\n        self.transformer = Transformer(dim, depth, heads, dim_head, ff_mult)\n        self.to_logits = n